⚡ Intermédiaire | ⏱ 60 min | 🔑 Concepts : pytest, assert, fixtures, parametrize, mocking

# 03 - Tests Unitaires avec pytest

## Objectifs

- Comprendre pourquoi les tests sont essentiels
- Maîtriser **pytest**, le framework de test moderne
- Utiliser les **fixtures** pour du setup/teardown réutilisable
- Paramétrer les tests avec **@pytest.mark.parametrize**
- Utiliser le **mocking** pour isoler le code testé
- Organiser une suite de tests professionnelle

## Prérequis

- Python 3.7+
- Connaissance de base de Python
- Comprendre les fonctions et les classes

## 1. Pourquoi tester ?

### Les 3 piliers des tests

#### 1. Confiance
Les tests vous donnent la **confiance** que votre code fonctionne comme prévu.

#### 2. Régression
Les tests détectent les **régressions** : quand un changement casse du code existant.

#### 3. Documentation
Les tests servent de **documentation vivante** : ils montrent comment utiliser votre code.

### Le coût de ne pas tester

- Bugs en production
- Peur de refactoriser
- Temps passé à déboguer manuellement
- Perte de confiance de l'équipe

### ROI des tests

```
Temps pour écrire un test    : 5-15 minutes
Temps pour déboguer en prod  : 2-8 heures
Coût d'un bug en production  : $$$$
```

## 2. pytest vs unittest

### unittest (bibliothèque standard)

```python
import unittest

class TestCalculatrice(unittest.TestCase):
    def test_addition(self):
        self.assertEqual(2 + 2, 4)
```

### pytest (moderne et pythonic)

```python
def test_addition():
    assert 2 + 2 == 4
```

### Pourquoi pytest ?

- ✅ Syntaxe simple : `assert` natif
- ✅ Pas besoin de classes
- ✅ Fixtures puissantes
- ✅ Plugins riches
- ✅ Meilleurs messages d'erreur
- ✅ Compatible avec unittest

In [ ]:
# Installation de pytest
!pip install pytest pytest-cov -q

## 3. Premier test avec pytest

In [ ]:
%%writefile calculatrice.py
"""Module calculatrice pour démonstration de tests."""

def addition(a, b):
    """Additionne deux nombres."""
    return a + b

def soustraction(a, b):
    """Soustrait b de a."""
    return a - b

def multiplication(a, b):
    """Multiplie deux nombres."""
    return a * b

def division(a, b):
    """Divise a par b."""
    if b == 0:
        raise ValueError("Division par zéro impossible")
    return a / b

In [ ]:
%%writefile test_calculatrice.py
"""Tests pour le module calculatrice."""
import pytest
from calculatrice import addition, soustraction, multiplication, division

def test_addition():
    """Test de l'addition."""
    assert addition(2, 3) == 5
    assert addition(-1, 1) == 0
    assert addition(0, 0) == 0

def test_soustraction():
    """Test de la soustraction."""
    assert soustraction(5, 3) == 2
    assert soustraction(0, 5) == -5

def test_multiplication():
    """Test de la multiplication."""
    assert multiplication(3, 4) == 12
    assert multiplication(-2, 3) == -6
    assert multiplication(0, 100) == 0

def test_division():
    """Test de la division."""
    assert division(10, 2) == 5
    assert division(7, 2) == 3.5

In [ ]:
# Exécuter les tests
!pytest test_calculatrice.py -v

### Convention de nommage

pytest découvre automatiquement les tests en suivant ces conventions :

- **Fichiers** : `test_*.py` ou `*_test.py`
- **Fonctions** : `def test_*():`
- **Classes** : `class Test*:`
- **Méthodes** : `def test_*():`

## 4. pytest.raises : Tester les exceptions

In [ ]:
%%writefile test_exceptions.py
"""Tests des exceptions."""
import pytest
from calculatrice import division

def test_division_par_zero():
    """Test que la division par zéro lève une exception."""
    with pytest.raises(ValueError):
        division(10, 0)

def test_division_par_zero_message():
    """Test que le message d'erreur est correct."""
    with pytest.raises(ValueError, match="Division par zéro"):
        division(10, 0)

def test_division_par_zero_detailed():
    """Test avec accès à l'exception."""
    with pytest.raises(ValueError) as exc_info:
        division(10, 0)
    
    assert "zéro" in str(exc_info.value)
    assert exc_info.type == ValueError

In [ ]:
!pytest test_exceptions.py -v

## 5. Fixtures : Setup/Teardown réutilisable

Les **fixtures** permettent de préparer l'environnement de test et de le nettoyer après.

In [ ]:
%%writefile test_fixtures.py
"""Démonstration des fixtures pytest."""
import pytest

# Fixture simple
@pytest.fixture
def nombres_exemple():
    """Fixture qui retourne une liste de nombres."""
    return [1, 2, 3, 4, 5]

# Fixture avec setup et teardown
@pytest.fixture
def fichier_temporaire():
    """Fixture qui crée et supprime un fichier temporaire."""
    import tempfile
    import os
    
    # Setup
    fd, path = tempfile.mkstemp()
    print(f"\nCréation fichier: {path}")
    
    # Le test utilise la fixture
    yield path
    
    # Teardown
    os.close(fd)
    os.unlink(path)
    print(f"\nSuppression fichier: {path}")

def test_somme(nombres_exemple):
    """Test utilisant la fixture nombres_exemple."""
    assert sum(nombres_exemple) == 15

def test_longueur(nombres_exemple):
    """Autre test utilisant la même fixture."""
    assert len(nombres_exemple) == 5

def test_fichier(fichier_temporaire):
    """Test utilisant la fixture fichier_temporaire."""
    with open(fichier_temporaire, 'w') as f:
        f.write("test data")
    
    with open(fichier_temporaire, 'r') as f:
        content = f.read()
    
    assert content == "test data"

In [ ]:
!pytest test_fixtures.py -v -s

### Scopes des fixtures

Les fixtures peuvent avoir différents scopes :

| Scope | Description | Utilisation |
|-------|-------------|-------------|
| `function` | Par défaut, recréée pour chaque test | Données de test simples |
| `class` | Une fois par classe de tests | Tests groupés |
| `module` | Une fois par fichier de test | Connexion DB (coûteuse) |
| `session` | Une fois pour toute la suite de tests | Configuration globale |

In [ ]:
%%writefile test_fixture_scopes.py
"""Démonstration des scopes de fixtures."""
import pytest

@pytest.fixture(scope="function")
def compteur_function():
    """Fixture function scope (défaut)."""
    print("\nSetup function fixture")
    return {"count": 0}

@pytest.fixture(scope="module")
def compteur_module():
    """Fixture module scope."""
    print("\nSetup module fixture (une fois)")
    return {"count": 0}

def test_un(compteur_function, compteur_module):
    compteur_function["count"] += 1
    compteur_module["count"] += 1
    print(f"Test 1 - function: {compteur_function['count']}, module: {compteur_module['count']}")

def test_deux(compteur_function, compteur_module):
    compteur_function["count"] += 1
    compteur_module["count"] += 1
    print(f"Test 2 - function: {compteur_function['count']}, module: {compteur_module['count']}")

In [ ]:
!pytest test_fixture_scopes.py -v -s

## 6. @pytest.mark.parametrize : Tester plusieurs cas

Plutôt que d'écrire plusieurs tests similaires, utilisez `parametrize` !

In [ ]:
%%writefile test_parametrize.py
"""Démonstration de @pytest.mark.parametrize."""
import pytest
from calculatrice import addition, division

# Test paramétré simple
@pytest.mark.parametrize("a,b,resultat_attendu", [
    (2, 3, 5),
    (0, 0, 0),
    (-1, 1, 0),
    (10, -5, 5),
    (100, 200, 300),
])
def test_addition_parametree(a, b, resultat_attendu):
    """Test de l'addition avec plusieurs cas."""
    assert addition(a, b) == resultat_attendu

# Test paramétré avec IDs personnalisés
@pytest.mark.parametrize("a,b,resultat_attendu", [
    (10, 2, 5),
    (7, 2, 3.5),
    (-10, 2, -5),
], ids=["entiers", "decimaux", "negatifs"])
def test_division_parametree(a, b, resultat_attendu):
    """Test de la division avec IDs personnalisés."""
    assert division(a, b) == resultat_attendu

# Parametrize multiple
@pytest.mark.parametrize("a", [1, 2, 3])
@pytest.mark.parametrize("b", [10, 20])
def test_combinaisons(a, b):
    """Test avec toutes les combinaisons de a et b."""
    # 3 valeurs * 2 valeurs = 6 tests
    assert addition(a, b) == a + b

In [ ]:
!pytest test_parametrize.py -v

## 7. Mocking : Isoler le code testé

Le **mocking** permet de remplacer des dépendances externes (API, base de données, fichiers) par des objets contrôlés.

In [ ]:
%%writefile api_client.py
"""Client API pour démonstration de mocking."""
import requests

def get_user_data(user_id):
    """Récupère les données d'un utilisateur depuis une API."""
    response = requests.get(f"https://api.example.com/users/{user_id}")
    response.raise_for_status()
    return response.json()

def get_user_name(user_id):
    """Récupère le nom d'un utilisateur."""
    data = get_user_data(user_id)
    return data.get("name", "Inconnu")

In [ ]:
%%writefile test_mocking.py
"""Tests avec mocking."""
import pytest
from unittest.mock import Mock, patch, MagicMock
from api_client import get_user_name, get_user_data

# Test avec patch pour remplacer requests.get
@patch('api_client.requests.get')
def test_get_user_data(mock_get):
    """Test avec mock de requests.get."""
    # Configurer le mock
    mock_response = Mock()
    mock_response.json.return_value = {"id": 1, "name": "Alice"}
    mock_get.return_value = mock_response
    
    # Appeler la fonction
    result = get_user_data(1)
    
    # Vérifications
    assert result == {"id": 1, "name": "Alice"}
    mock_get.assert_called_once_with("https://api.example.com/users/1")

# Test avec patch d'une fonction interne
@patch('api_client.get_user_data')
def test_get_user_name(mock_get_user_data):
    """Test avec mock de get_user_data."""
    # Configurer le mock
    mock_get_user_data.return_value = {"id": 1, "name": "Bob"}
    
    # Appeler la fonction
    result = get_user_name(1)
    
    # Vérifications
    assert result == "Bob"
    mock_get_user_data.assert_called_once_with(1)

# Test avec patch comme context manager
def test_get_user_name_context_manager():
    """Test avec patch en context manager."""
    with patch('api_client.get_user_data') as mock_func:
        mock_func.return_value = {"id": 2, "name": "Charlie"}
        
        result = get_user_name(2)
        
        assert result == "Charlie"

# Test avec side_effect pour simuler des exceptions
@patch('api_client.requests.get')
def test_get_user_data_error(mock_get):
    """Test avec simulation d'erreur."""
    mock_get.side_effect = requests.exceptions.RequestException("Network error")
    
    with pytest.raises(requests.exceptions.RequestException):
        get_user_data(999)

# Import requests pour le test
import requests

In [ ]:
!pip install requests -q
!pytest test_mocking.py -v

### Principales fonctions de mocking

| Fonction | Usage |
|----------|-------|
| `Mock()` | Crée un objet mock simple |
| `MagicMock()` | Mock avec méthodes magiques (__str__, __len__, etc.) |
| `patch()` | Remplace temporairement un objet |
| `return_value` | Valeur retournée par le mock |
| `side_effect` | Exception ou liste de valeurs à retourner |
| `assert_called()` | Vérifie que le mock a été appelé |
| `assert_called_once()` | Vérifie qu'il a été appelé une seule fois |
| `assert_called_with()` | Vérifie les arguments d'appel |

## 8. Markers : Organisation des tests

Les **markers** permettent de catégoriser et filtrer les tests.

In [ ]:
%%writefile test_markers.py
"""Démonstration des markers pytest."""
import pytest
import time

@pytest.mark.slow
def test_operation_lente():
    """Test marqué comme lent."""
    time.sleep(0.1)
    assert True

@pytest.mark.fast
def test_operation_rapide():
    """Test marqué comme rapide."""
    assert 1 + 1 == 2

@pytest.mark.skip(reason="Fonctionnalité pas encore implémentée")
def test_future_feature():
    """Test à ignorer."""
    assert False

@pytest.mark.skipif(pytest.__version__ < "7.0", reason="Nécessite pytest 7+")
def test_nouvelle_fonctionnalite():
    """Test conditionnel."""
    assert True

@pytest.mark.xfail(reason="Bug connu #123")
def test_avec_bug_connu():
    """Test qui devrait échouer (xfail = expected failure)."""
    assert 1 / 0  # On sait que ça échoue

@pytest.mark.integration
def test_integration_database():
    """Test d'intégration (marker custom)."""
    assert True

In [ ]:
# Exécuter tous les tests
!pytest test_markers.py -v

In [ ]:
# Exécuter seulement les tests rapides
!pytest test_markers.py -v -m fast

In [ ]:
# Exécuter sauf les tests lents
!pytest test_markers.py -v -m "not slow"

## 9. conftest.py : Partager des fixtures

Le fichier `conftest.py` permet de définir des fixtures partagées entre plusieurs fichiers de tests.

In [ ]:
%%writefile conftest.py
"""Configuration partagée pour tous les tests."""
import pytest

@pytest.fixture(scope="session")
def config():
    """Configuration globale pour les tests."""
    return {
        "api_url": "https://api.test.example.com",
        "timeout": 30,
        "debug": True
    }

@pytest.fixture
def sample_data():
    """Données d'exemple pour les tests."""
    return [
        {"id": 1, "name": "Alice", "age": 30},
        {"id": 2, "name": "Bob", "age": 25},
        {"id": 3, "name": "Charlie", "age": 35},
    ]

@pytest.fixture
def mock_database():
    """Simulation d'une base de données."""
    class MockDB:
        def __init__(self):
            self.data = {}
        
        def insert(self, key, value):
            self.data[key] = value
        
        def get(self, key):
            return self.data.get(key)
        
        def clear(self):
            self.data.clear()
    
    db = MockDB()
    yield db
    db.clear()  # Cleanup

# Configuration des markers custom
def pytest_configure(config):
    config.addinivalue_line(
        "markers", "slow: marks tests as slow (deselect with '-m \"not slow\"')"
    )
    config.addinivalue_line(
        "markers", "fast: marks tests as fast"
    )
    config.addinivalue_line(
        "markers", "integration: marks tests as integration tests"
    )

In [ ]:
%%writefile test_with_conftest.py
"""Tests utilisant les fixtures de conftest.py."""

def test_config(config):
    """Test utilisant la fixture config."""
    assert config["api_url"] == "https://api.test.example.com"
    assert config["timeout"] == 30

def test_sample_data(sample_data):
    """Test utilisant la fixture sample_data."""
    assert len(sample_data) == 3
    assert sample_data[0]["name"] == "Alice"

def test_mock_database(mock_database):
    """Test utilisant la fixture mock_database."""
    mock_database.insert("user:1", {"name": "Test User"})
    user = mock_database.get("user:1")
    assert user["name"] == "Test User"

In [ ]:
!pytest test_with_conftest.py -v

## Pièges courants

### 1. Tests couplés (dépendants les uns des autres)

```python
# ❌ MAUVAIS : Les tests partagent un état
compteur = 0

def test_un():
    global compteur
    compteur += 1
    assert compteur == 1

def test_deux():
    global compteur
    compteur += 1
    assert compteur == 2  # Échoue si test_un n'est pas exécuté avant

# ✅ BON : Chaque test est indépendant
@pytest.fixture
def compteur():
    return 0

def test_un(compteur):
    compteur += 1
    assert compteur == 1

def test_deux(compteur):
    compteur += 1
    assert compteur == 1  # Toujours OK
```

### 2. Fixtures trop complexes

```python
# ❌ MAUVAIS : Fixture qui fait trop de choses
@pytest.fixture
def setup_everything():
    # Crée DB, API, fichiers, etc.
    # 100 lignes de code...
    pass

# ✅ BON : Fixtures modulaires
@pytest.fixture
def database():
    # ...

@pytest.fixture
def api_client():
    # ...

@pytest.fixture
def test_files():
    # ...
```

### 3. Mocker trop (tester l'implémentation)

```python
# ❌ MAUVAIS : Trop de mocking
@patch('module.function_a')
@patch('module.function_b')
@patch('module.function_c')
def test_complex(mock_c, mock_b, mock_a):
    # On teste les mocks, pas le code !
    pass

# ✅ BON : Mocker seulement les dépendances externes
@patch('module.external_api_call')
def test_logic(mock_api):
    # Tester la logique métier
    pass
```

### 4. Ne pas tester les cas limites

```python
# ❌ MAUVAIS : Tester seulement le cas nominal
def test_division():
    assert division(10, 2) == 5

# ✅ BON : Tester les cas limites
@pytest.mark.parametrize("a,b,expected", [
    (10, 2, 5),      # Cas normal
    (0, 5, 0),       # Zéro au numérateur
    (10, 3, 10/3),   # Résultat décimal
    (-10, 2, -5),    # Négatif
])
def test_division_complete(a, b, expected):
    assert division(a, b) == expected

def test_division_par_zero():
    with pytest.raises(ValueError):
        division(10, 0)
```

## Mini-Exercices

### Exercice 1 : Tester une fonction de validation

Écrivez des tests pour cette fonction de validation d'email :

In [ ]:
%%writefile email_validator.py
import re

def valider_email(email):
    """Valide un email simple."""
    if not email:
        raise ValueError("Email vide")
    
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    if not re.match(pattern, email):
        raise ValueError("Email invalide")
    
    return True

In [ ]:
# Écrivez vos tests ici
# Testez au moins :
# - Emails valides
# - Emails invalides
# - Email vide
# - None


### Exercice 2 : Fixture pour une base de données mock

Créez une fixture qui simule une base de données avec insert, get, delete :

In [ ]:
# Votre fixture ici


### Exercice 3 : Tests paramétrés

Écrivez un test paramétré pour cette fonction :

In [ ]:
%%writefile string_utils.py
def est_palindrome(texte):
    """Vérifie si un texte est un palindrome (ignore la casse et les espaces)."""
    texte_nettoye = texte.lower().replace(" ", "")
    return texte_nettoye == texte_nettoye[::-1]

In [ ]:
# Écrivez un test paramétré avec au moins 5 cas de test


## Solutions

### Solution Exercice 1

In [ ]:
%%writefile test_email_validator.py
import pytest
from email_validator import valider_email

@pytest.mark.parametrize("email", [
    "user@example.com",
    "john.doe@company.fr",
    "test+tag@domain.co.uk",
    "user123@test-domain.com",
])
def test_emails_valides(email):
    """Test des emails valides."""
    assert valider_email(email) is True

@pytest.mark.parametrize("email", [
    "invalid",
    "@example.com",
    "user@",
    "user @example.com",
    "user@example",
])
def test_emails_invalides(email):
    """Test des emails invalides."""
    with pytest.raises(ValueError, match="Email invalide"):
        valider_email(email)

def test_email_vide():
    """Test avec email vide."""
    with pytest.raises(ValueError, match="Email vide"):
        valider_email("")

def test_email_none():
    """Test avec None."""
    with pytest.raises(ValueError, match="Email vide"):
        valider_email(None)

In [ ]:
!pytest test_email_validator.py -v

### Solution Exercice 2

In [ ]:
%%writefile test_database_fixture.py
import pytest

@pytest.fixture
def mock_database():
    """Fixture simulant une base de données."""
    class MockDatabase:
        def __init__(self):
            self.storage = {}
        
        def insert(self, key, value):
            if key in self.storage:
                raise ValueError(f"Clé {key} existe déjà")
            self.storage[key] = value
        
        def get(self, key):
            if key not in self.storage:
                raise KeyError(f"Clé {key} introuvable")
            return self.storage[key]
        
        def delete(self, key):
            if key not in self.storage:
                raise KeyError(f"Clé {key} introuvable")
            del self.storage[key]
        
        def clear(self):
            self.storage.clear()
    
    db = MockDatabase()
    yield db
    db.clear()

def test_insert(mock_database):
    """Test de l'insertion."""
    mock_database.insert("key1", "value1")
    assert mock_database.get("key1") == "value1"

def test_get_inexistant(mock_database):
    """Test get sur clé inexistante."""
    with pytest.raises(KeyError):
        mock_database.get("nonexistent")

def test_delete(mock_database):
    """Test de suppression."""
    mock_database.insert("key1", "value1")
    mock_database.delete("key1")
    with pytest.raises(KeyError):
        mock_database.get("key1")

def test_insert_duplicate(mock_database):
    """Test insertion de doublon."""
    mock_database.insert("key1", "value1")
    with pytest.raises(ValueError):
        mock_database.insert("key1", "value2")

In [ ]:
!pytest test_database_fixture.py -v

### Solution Exercice 3

In [ ]:
%%writefile test_palindrome.py
import pytest
from string_utils import est_palindrome

@pytest.mark.parametrize("texte,attendu", [
    ("radar", True),
    ("kayak", True),
    ("A man a plan a canal Panama", True),
    ("Was it a car or a cat I saw", True),
    ("Engage le jeu que je le gagne", True),
    ("hello", False),
    ("python", False),
    ("", True),  # Chaîne vide est un palindrome
    ("a", True),  # Un seul caractère
], ids=[
    "radar",
    "kayak",
    "panama",
    "cat_or_car",
    "francais",
    "non_palindrome_hello",
    "non_palindrome_python",
    "vide",
    "un_caractere",
])
def test_palindrome(texte, attendu):
    """Test de la fonction est_palindrome."""
    assert est_palindrome(texte) == attendu

In [ ]:
!pytest test_palindrome.py -v